# 🚀 Koushole RAG Processor (PaddleOCR Edition)

**Process NCTB textbooks for RAG using FREE cloud GPU**

**Version:** PaddleOCR (Fixed Dependencies)

---

**SETUP:**
1. Go to Runtime → Change runtime type → Select **T4 GPU**
2. Click **Runtime → Restart session** (CRITICAL)
3. Run all cells in order
4. Enter API keys when prompted

In [ ]:
#@title 1️⃣ Install Dependencies
# Install PaddlePaddle GPU version first
!pip install -q paddlepaddle-gpu
!pip install -q paddleocr pdf2image Pillow supabase voyageai
# CRITICAL: Uninstall langchain to prevent PaddleOCR from trying to import missing modules
!pip uninstall -y langchain langchain-community
!apt-get install -q poppler-utils
print("✅ Dependencies installed!")

In [ ]:
#@title 2️⃣ Enter Credentials
from getpass import getpass

print("🔑 Enter your credentials (hidden):")
SUPABASE_URL = input("Supabase URL: ")
SUPABASE_KEY = getpass("Supabase Service Key: ")
VOYAGE_API_KEY = getpass("Voyage AI API Key: ")

if not SUPABASE_URL or not SUPABASE_KEY or not VOYAGE_API_KEY:
    raise ValueError("❌ All credentials are required!")
print("✅ Credentials saved.")

In [ ]:
#@title 3️⃣ Initialize Clients & Models
from supabase import create_client
import voyageai
import sys
from paddleocr import PaddleOCR

# 1. Setup Clients
try:
    supabase = create_client(SUPABASE_URL, SUPABASE_KEY)
    voyage = voyageai.Client(api_key=VOYAGE_API_KEY)
    print("✅ Clients initialized")
except Exception as e:
    print(f"❌ Client error: {e}")
    sys.exit(1)

# 2. Setup OCR
print("\n🔮 Setting up PaddleOCR (GPU)...")
try:
    # Initialize PaddleOCR for multilingual (includes Bangla)
    # use_angle_cls=True for rotated text detection
    # lang='en' is used as base but it handles multilingual well
    ocr = PaddleOCR(use_angle_cls=True, lang='en', use_gpu=True, show_log=False)
    print("✅ PaddleOCR Model Loaded Successfully")

except Exception as e:
    print(f"❌ Model Loading Error: {e}")
    sys.exit(1)

In [ ]:
#@title 4️⃣ Helper Functions
from pdf2image import convert_from_bytes
import requests
from tqdm import tqdm
from PIL import Image
import numpy as np
import time

def extract_text_paddleocr(pdf_bytes):
    """Extract text from PDF using PaddleOCR"""
    try:
        # Convert PDF to images (dpi=200 for better OCR accuracy)
        images = convert_from_bytes(pdf_bytes, dpi=200)
        all_text = []
        
        pbar = tqdm(total=len(images), desc="OCR Processing")
        for img in images:
            # Convert PIL image to numpy array
            img_np = np.array(img)
            
            # Run OCR
            result = ocr.ocr(img_np, cls=True)
            
            # Extract text from result
            page_lines = []
            if result and result[0]:
                for line in result[0]:
                    if line[1]:  # Check if text exists
                        text = line[1][0]  # Get the text string
                        page_lines.append(text)
            
            page_text = "\n".join(page_lines)
            all_text.append(page_text)
            
            pbar.update(1)
            
        pbar.close()
        return "\n\n".join(all_text)
        
    except Exception as e:
        print(f"OCR Failed: {e}")
        return ""

def chunk_text(text, chunk_size=2000, overlap=200):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        if chunk.strip():
            chunks.append(chunk.strip())
        start = end - overlap
        if start >= len(text) - overlap:
            break
    return [c for c in chunks if len(c) > 50]

def generate_embeddings(chunks):
    """Generate embeddings with rate limit handling (3 RPM for free tier)"""
    if not chunks: return []
    embeddings = []
    batch_size = 20
    total_batches = (len(chunks) + batch_size - 1) // batch_size
    print(f'  ⏳ Rate limit mode: {total_batches} batches @ 30s delay each')
    
    for i in tqdm(range(0, len(chunks), batch_size), desc="Generating Embeddings"):
        batch = [c[:8000] for c in chunks[i:i+batch_size]]
        res = voyage.embed(batch, model="voyage-multilingual-2", input_type="document")
        embeddings.extend(res.embeddings)
        # Wait 30 seconds between batches to stay under 3 RPM limit
        if i + batch_size < len(chunks):
            time.sleep(30)
    return embeddings

print("✅ Helper functions ready!")

In [ ]:
#@title 5️⃣ Fetch Pending Books
print("📡 Fetching pending books from Supabase...")

# Get library books
lib_res = supabase.table('library_books').select('id, title, file_url').or_('chunks_generated.is.null,chunks_generated.eq.false').execute()
lib_books = lib_res.data or []

# Get official books
off_res = supabase.table('official_resources').select('id, title, file_url').or_('chunks_generated.is.null,chunks_generated.eq.false').execute()
off_books = off_res.data or []

all_books = [(b, 'library') for b in lib_books] + [(b, 'official') for b in off_books]

print(f"📚 Library books: {len(lib_books)}")
print(f"📖 Official books: {len(off_books)}")
print(f"📊 Total to process: {len(all_books)}")

In [ ]:
#@title 6️⃣ Process Loop 🚀
import time

def process_book(book, source_type):
    title = book['title']
    print(f"\n📘 Processing: {title}")
    
    # 1. Download
    try:
        print("  📥 Downloading...")
        r = requests.get(book['file_url'])
        if r.status_code != 200:
            print("  ❌ Download failed")
            return False
        pdf_bytes = r.content
        print(f"  📄 Size: {len(pdf_bytes) / 1024 / 1024:.2f} MB")
    except Exception as e:
        print(f"  ❌ Download error: {e}")
        return False

    # 2. OCR (Using PaddleOCR)
    start_time = time.time()
    text = extract_text_paddleocr(pdf_bytes)
    ocr_time = time.time() - start_time
    
    if not text or len(text) < 100:
        print(f"  ❌ OCR extracted too little text ({len(text)} chars)")
        return False
    print(f"  ✅ Extracted {len(text)} characters in {ocr_time:.1f}s")

    # 3. Chunk
    chunks = chunk_text(text)
    print(f"  📦 Generated {len(chunks)} chunks")

    # 4. Embed
    try:
        embeddings = generate_embeddings(chunks)
    except Exception as e:
        print(f"  ❌ Embedding error: {e}")
        return False

    # 5. Store
    print("  💾 saving to databse...")
    id_col = 'library_book_id' if source_type == 'library' else 'resource_id'
    
    try:
        # Clear old chunks
        supabase.table('book_chunks').delete().eq(id_col, book['id']).execute()
        
        # Insert NEW chunks
        batch = []
        for idx, (txt, emb) in enumerate(zip(chunks, embeddings)):
            batch.append({
                id_col: book['id'],
                'chunk_index': idx,
                'chunk_text': txt,
                'embedding': emb
            })
            if len(batch) >= 50:
                supabase.table('book_chunks').insert(batch).execute()
                batch = []
        if batch:
            supabase.table('book_chunks').insert(batch).execute()
            
        # Update Status
        table = 'library_books' if source_type == 'library' else 'official_resources'
        supabase.table(table).update({
            'chunks_generated': True,
            'total_chunks': len(chunks),
            'is_processed': True
        }).eq('id', book['id']).execute()
        
        print("  ✅ SUCCESS!")
        return True
        
    except Exception as e:
        print(f"  ❌ Database error: {e}")
        return False

# --- MAIN LOOP ---
success_count = 0
fail_count = 0

for book, source in all_books:
    if process_book(book, source):
        success_count += 1
    else:
        fail_count += 1
    time.sleep(1)

print(f"\n{'='*40}")
print(f"🏁 DONE! Success: {success_count}, Failed: {fail_count}")

## 🎉 All Done!

Check your website chat - the new books should now be searchable!